# Dynamic Z-Score Calculation for Stock Price Ratios
The z-score is a statistical measurement that describes a value's relationship to the mean of a group of values. It is measured in terms of standard deviations from the mean. If a z-score is 0, it indicates that the data point's score is identical to the mean score. A z-score of 1.0 would indicate a value that is one standard deviation from the mean. Z-scores may be positive or negative, with a positive value indicating the score is above the mean and a negative score indicating it is below the mean.

In this case, we're calculating the z-score for the ratio of two stock prices with a rolling window. The formula for the z-score is:

$$ Z = \frac{X - MA}{MSD} $$

Where:
- $Z$ is the z-score,
- $X$ is the value of the element (in this case, the current price ratio),
- $MA$ is the shifted moving average of the price ratio over a dynamic window (e.g., Shifted Moving Average @ 3 days or 5 days),
- $MSD$ is the shifted moving standard deviation of the price ratio over the same window.

Consider the following table which shows the price of "Sherlock Holmes LLC" and "Conan Doyle GmbH", the price ratio, the shifted 3-day and 5-day moving average and moving standard deviation for 10 days:

| Day | Sherlock Holmes LLC Price | Conan Doyle GmbH Price | Price Ratio | Shifted 3-day MA | Shifted 3-day MSD | Z-Score (3-day) | Shifted 5-day MA | Shifted 5-day MSD | Z-Score (5-day) |
|-----|---------------------------|------------------------|-------------|------------------|-------------------|-----------------|------------------|-------------------|-----------------|
| 1   | 100                       | 200                    | 0.5         | -                | -                 | -               | -                | -                 | -               |
| 2   | 105                       | 210                    | 0.5         | -                | -                 | -               | -                | -                 | -               |
| 3   | 110                       | 205                    | 0.54        | -                | -                 | -               | -                | -                 | -               |
| 4   | 115                       | 215                    | 0.53        | 0.512            | 0.021             | 1.074           | -                | -                 | -               |
| 5   | 120                       | 220                    | 0.55        | 0.524            | 0.021             | 1.048           | -                | -                 | -               |
| 6   | 125                       | 225                    | 0.56        | 0.538            | 0.006             | 2.921           | 0.523            | 0.022             | 1.481           |
| 7   | 130                       | 230                    | 0.57        | 0.545            | 0.010             | 1.927           | 0.534            | 0.021             | 1.465           |
| 8   | 135                       | 235                    | 0.57        | 0.555            | 0.010             | 1.929           | 0.548            | 0.013             | 2.094           |
| 9   | 140                       | 240                    | 0.58        | 0.565            | 0.010             | 1.930           | 0.555            | 0.016             | 1.803           |
| 10  | 145                       | 245                    | 0.59        | 0.574            | 0.010             | 1.932           | 0.565            | 0.015             | 1.805           |

In this example, the price ratio varies slightly each day, so the shifted moving averages and moving standard deviations also vary, and the z-score for each day will be different. In a real-world scenario, these values would vary even more from day to day.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, widgets, Output
from IPython.display import display, clear_output

# Fetch historical data for MSFT and GOOGL
start_date = '2021-01-01'
end_date = '2023-01-01'
msft = yf.download('MSFT', start=start_date, end=end_date, progress=False)
googl = yf.download('GOOGL', start=start_date, end=end_date, progress=False)

# Create a DataFrame with the adjusted close prices
df = pd.DataFrame({'MSFT': msft['Adj Close'], 'GOOGL': googl['Adj Close']})

# Function to calculate Z-score and mean spread for a given window
def calculate_z_score(df, window):
    spread = df['MSFT'] - df['GOOGL']
    mean_spread = spread.rolling(window=window).mean()
    std_spread = spread.rolling(window=window).std()
    z_score = (spread - mean_spread) / std_spread
    z_score = z_score.clip(lower=-5, upper=5)  # Clip Z-score values to range [-5, 5]
    return z_score.round(1)  # Round Z-score values to one decimal place

# Function to generate trade signals based on Z-score
def generate_trade_signals(df, z_threshold):
    signals = pd.DataFrame(index=df.index)
    signals['Z_Score'] = df['Z_Score']
    signals['Position'] = 0  # 0: neutral, 1: long MSFT, -1: short MSFT
    signals['Signal'] = np.nan
    
    for i in range(1, len(signals)):
        if signals['Z_Score'].iloc[i] > z_threshold and signals['Position'].iloc[i-1] != -1:
            signals.at[signals.index[i], 'Position'] = -1  # Short MSFT, Long GOOGL
            signals.at[signals.index[i], 'Signal'] = -1
        elif signals['Z_Score'].iloc[i] < -z_threshold and signals['Position'].iloc[i-1] != 1:
            signals.at[signals.index[i], 'Position'] = 1  # Long MSFT, Short GOOGL
            signals.at[signals.index[i], 'Signal'] = 1
        elif -1.3 < signals['Z_Score'].iloc[i] < 1.3 and signals['Position'].iloc[i-1] != 0:
            signals.at[signals.index[i], 'Position'] = 0  # Close position
            signals.at[signals.index[i], 'Signal'] = 0
        else:
            signals.at[signals.index[i], 'Position'] = signals['Position'].iloc[i-1]

    signals['Close_Position'] = (signals['Signal'] == 0)
    
    return signals

# Calculate returns based on the generated signals
def calculate_returns(df, signals, initial_investment):
    df['Position'] = signals['Position']
    df['Position'] = df['Position'].shift(1).fillna(0)  # Shift positions to align with returns
    df['Returns'] = df['MSFT'].pct_change() * df['Position']
    df['Cumulative_Returns'] = (1 + df['Returns']).cumprod() * initial_investment
    return df

# Create the output widget for displaying the plot
plot_output = Output()

# Display the empty output widget once
display(plot_output)

# Function to update the graph
def update_graph(window, z_threshold):
    with plot_output:
        # Clear the previous output
        clear_output(wait=True)
        
        df['Z_Score'] = calculate_z_score(df, window)
        signals = generate_trade_signals(df, z_threshold)
        df_updated = calculate_returns(df, signals, initial_investment)
        filtered_signals_updated = signals.dropna(subset=['Signal'])
        
        # Create subplots
        fig = make_subplots(
            rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, 
            specs=[[{"secondary_y": True}], [{}], [{}]], 
            subplot_titles=("Stock Prices", "Z-Scores and Trade Signals", "Cumulative Returns")
        )
        
        # Add traces for stock prices with two y-axes
        fig.add_trace(go.Scatter(x=df.index, y=df['MSFT'], mode='lines', name='MSFT', line=dict(color='blue')), row=1, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df['GOOGL'], mode='lines', name='GOOGL', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        
        # Add trace for Z-score
        fig.add_trace(go.Scatter(x=df.index, y=df['Z_Score'], mode='lines', name='Z-Score'), row=2, col=1)
        
        # Add trade signal markers
        long_signals = filtered_signals_updated[filtered_signals_updated['Signal'] == 1]
        short_signals = filtered_signals_updated[filtered_signals_updated['Signal'] == -1]
        close_signals = filtered_signals_updated[filtered_signals_updated['Close_Position']]
        
        fig.add_trace(go.Scatter(x=long_signals.index, y=long_signals['Z_Score'], mode='markers', name='Long MSFT, Short GOOGL', marker=dict(symbol='triangle-up', color='green', size=10)), row=2, col=1)
        fig.add_trace(go.Scatter(x=short_signals.index, y=short_signals['Z_Score'], mode='markers', name='Short MSFT, Long GOOGL', marker=dict(symbol='triangle-down', color='red', size=10)), row=2, col=1)
        fig.add_trace(go.Scatter(x=close_signals.index, y=close_signals['Z_Score'], mode='markers', name='Close Position', marker=dict(symbol='x', color='yellow', size=10)), row=2, col=1)
        
        # Add trace for cumulative returns
        fig.add_trace(go.Scatter(x=df.index, y=df_updated['Cumulative_Returns'], mode='lines', name='Cumulative Returns', line=dict(color='purple')), row=3, col=1)
        
        # Update layout
        fig.update_layout(
            title=f'Pairs Trading Strategy with Window {window} and Z-Score Threshold {z_threshold}',
            height=800,
            template='plotly_dark'
        )
        
        # Update y-axes titles
        fig.update_yaxes(title_text="MSFT Price", secondary_y=False, row=1, col=1)
        fig.update_yaxes(title_text="GOOGL Price", secondary_y=True, row=1, col=1)
        fig.update_yaxes(title_text="Z-Score", range=[-5, 5], row=2, col=1)
        fig.update_yaxes(title_text="Cumulative Returns", row=3, col=1)
        
        fig.show()

# Create dropdown widgets for window and Z-score threshold
window_dropdown = widgets.Dropdown(
    options=[(f'Window {window}', window) for window in [10, 20, 30, 50]],
    value=20,
    description='Window:',
)

z_threshold_dropdown = widgets.Dropdown(
    options=[(f'Z-Score Threshold {z_threshold}', z_threshold) for z_threshold in np.arange(2.0, 4.1, 0.1).round(1)],
    value=2.0,
    description='Z-Score Threshold:',
)

# Create interactive widget
initial_investment = 10000
interact(update_graph, window=window_dropdown, z_threshold=z_threshold_dropdown)


Output()

interactive(children=(Dropdown(description='Window:', index=1, options=(('Window 10', 10), ('Window 20', 20), …

<function __main__.update_graph(window, z_threshold)>